# Ingest CSV files in entwine
This notebook was run for each dataset (defined by the input path) and executes following steps:
* Ingest the pre-processed water column data files into entwine point tiles (EPT) which are octtree data structures that allow for fast location querying of the data.
* Optionally visualizes the ingested dataset in Potree Viewer.

## Set the input parameters
* input_path: the folder where all preprocessed files to be ingested are located. MBES Data available on request.
* entwine_path: the folder to where the entwine point tiles need to be written. MBES Data available on request.
* in_srs:  EPSG code of the Coordinate Reference System of the original point data (see https://epsg.io)
* out_srs: EPSG code of the Coordinate Reference System of the entwine point tiles (see https://epsg.io)

In [1]:
input_path = 'data/tmp/20210204_VLIZ_TIMBERS'
entwine_path = 'Multiingest_0095_6565_21092'
in_srs = "EPSG:4326"
out_srs =  "EPSG:32631"

Create folders if they don't exist

In [2]:
from timbers_code.utils import create_folder_if_absent
create_folder_if_absent(entwine_path)

## Write an entwine configuration file

Based on https://entwine.io/configuration.html.


In [3]:
import json
entwine_config_file = entwine_path +'/entwine_config_rgb.json'
entwine_config = {
    "input": input_path,
    "output": entwine_path,
    "reprojection": {
        "in": in_srs,
        "out": out_srs
    },  
    "schema": [
        { "name": "X", "type": "double" },
        { "name": "Y", "type": "double" },
        { "name": "Z", "type": "double" }, #        { "name": "GpsTime", "type": "double" },   
        { "name": "value_db", "type": "double" },
        { "name": "Red", "type": "uint8" },
        { "name": "Green", "type": "uint8" },
        { "name": "Blue", "type": "uint8" },
        { "name": "Alpha", "type": "uint8" }
    ],
    "dataType": "binary",
    "threads": [12, 2]
    
}
with open(entwine_config_file, 'w', encoding='utf-8') as f:
    json.dump(entwine_config, f, ensure_ascii=False, indent=4)

## Run entwine build with the config file to ingest the data
In case of multiple subsets, also run a merge on the entwine path

In [ ]:
%%time
!entwine build -c $entwine_config_file

## Visualize the data using Potree Viewer
* Run following line in the anaconda prompt (pcp-env) `http-server data -p 8080 --cors` 
* Open a webpage at http://potree.entwine.io/data/view.html?r=http://localhost:8080 + path to entwine